In [1]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import pandas as pd
import torchvision.models as models 
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
import matplotlib.pyplot as plt
import latent_plot as lp
from torch.autograd import Variable
from torch.optim import lr_scheduler
import seaborn as sns
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import copy
#from model import VAE

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from data_loader import *
from model import convVAE
from sklearn.manifold import TSNE

In [2]:
#only on validation set
csv_path = '/home/vip/sayan-mandal/datasets/obj_criteria/20200619-objective.csv'
img_path = '/home/vip/sayan-mandal/datasets/obj_criteria/good_reduced/'

ResizeParam = 128
FD = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.RandomHorizontalFlip(p=0.5) ,transforms.ToTensor()]))
FD2 = FullDataLoader(csv_path, img_path,transform=transforms.Compose([transforms.Resize(ResizeParam), transforms.ToTensor()]))

#random sampler
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(FD)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
val_indices =  indices[:split]

# Creating PT data samplers and loaders:
valid_sampler = SubsetRandomSampler(val_indices)

#train_loader = torch.utils.data.DataLoader(FD, batch_size=batch_size, sampler=train_sampler)
data_loader = torch.utils.data.DataLoader(FD2, batch_size=batch_size, sampler=valid_sampler)

In [66]:
#vae model structure
nl = 1024
model = convVAE((3,128,128),nl).cuda()
model.load_state_dict(torch.load('/home/vip/sayan-mandal/vae-convmodels/dfcvae_inp128_nl'+str(nl)+'_a1b1.torch'))
print(model)

convVAE(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2d(64, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Sequential(
    (0): Linear(in_features=16384, out_features=4096, bias=True)
    (1): BatchNo

In [67]:
for i, (data,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10) in enumerate(data_loader):
    y, _,_ =  model.encode(Variable(data).cuda())
    y = y.cpu().detach().numpy()
    if i == 0:
        yd = y
        lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10 = l1.numpy(),l2.numpy(),l3.numpy(),l4.numpy(),l5.numpy(),l6.numpy(),l7.numpy(),l8.numpy(),l9.numpy(),l10.numpy()
    else:
        yd = np.append(yd,y,axis=0)
        lbl1 = np.append(lbl1,l1.numpy(), axis=0)
        lbl2= np.append(lbl2,l2.numpy(), axis=0)
        lbl3= np.append(lbl3,l3.numpy(), axis=0)
        lbl4= np.append(lbl4,l4.numpy(), axis=0)
        lbl5= np.append(lbl5,l5.numpy(), axis=0)
        lbl6= np.append(lbl6,l6.numpy(), axis=0)
        lbl7= np.append(lbl7,l7.numpy(), axis=0)
        lbl8= np.append(lbl8,l8.numpy(), axis=0)
        lbl9= np.append(lbl9,l9.numpy(), axis=0)
        lbl10 = np.append(lbl10,l10.numpy(), axis=0)

In [68]:
yd.shape

(1725, 1024)

In [69]:
lbl1 = lbl1.reshape(-1,1)
lbl2 = lbl2.reshape(-1,1)
lbl3 = lbl3.reshape(-1,1)
lbl4 = lbl4.reshape(-1,1)
lbl5 = lbl5.reshape(-1,1)
lbl6 = lbl6.reshape(-1,1)
lbl7 = lbl7.reshape(-1,1)
lbl8 = lbl8.reshape(-1,1)
lbl9 = lbl9.reshape(-1,1)
lbl10 = lbl10.reshape(-1,1)

In [70]:
cols = ['class','mdprob','psdprob','md_inf','md_sup','rnfl_g','rnfl_ti','rnfl_ni','rnfl_ts','rnfl_ns']
for i in range(yd.shape[1]):
    cols += ['ls_'+str(i)]

In [71]:
to_df = np.concatenate((lbl1,lbl2,lbl3,lbl4,lbl5,lbl6,lbl7,lbl8,lbl9,lbl10,yd),axis=1)

In [72]:
df = pd.DataFrame(data=to_df, columns=cols)

In [73]:
df

,class,mdprob,psdprob,md_inf,md_sup,rnfl_g,rnfl_ti,rnfl_ni,rnfl_ts,rnfl_ns,...,ls_1014,ls_1015,ls_1016,ls_1017,ls_1018,ls_1019,ls_1020,ls_1021,ls_1022,ls_1023
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.046689,-0.025016,-0.026628,-0.020892,0.027162,0.029632,0.040702,0.066075,-0.004427,0.000321
1,1.0,5.0,5.0,4.0,4.0,2.0,2.0,1.0,2.0,1.0,...,0.032925,-0.099099,-0.063378,-0.038826,0.051563,0.046576,0.057624,0.028114,-0.126576,0.024889
2,1.0,5.0,5.0,4.0,4.0,2.0,2.0,0.0,2.0,0.0,...,0.004649,-0.062196,-0.028863,-0.032401,0.034031,-0.014232,0.027512,-0.035264,-0.026377,0.070742
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011080,0.045014,0.033053,0.021110,-0.086939,0.023849,0.065896,-0.012412,0.019625,-0.065533
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.014498,-0.006322,0.013759,0.040839,0.016941,-0.022691,0.036829,0.035225,-0.039381,-0.080854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.066701,-0.034549,-0.006745,0.062487,0.003613,0.025858,-0.086082,-0.042995,-0.041472,0.002556
1721,1.0,5.0,5.0,4.0,4.0,2.0,0.0,0.0,2.0,2.0,...,-0.024182,-0.024205,0.085697,0.086132,-0.079396,0.002030,-0.006185,0.012259,0.158076,0.094421
1722,1.0,1.0,3.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,...,0.020294,0.012979,-0.099042,-0.041777,0.029980,-0.006939,-0.054036,-0.005635,-0.037597,-0.055085
1723,1.0,5.0,5.0,4.0,4.0,2.0,2.0,0.0,2.0,1.0,...,0.008471,-0.040812,0.112429,0.007155,-0.018250,-0.009692,0.008074,0.050503,0.088690,0.036620


In [74]:
df.to_csv('latentdat_'+str(nl)+'.csv',index=False)

In [7]:
df = pd.read_csv('filelistandlabels.csv')

In [6]:
df.to_csv('filelistandlabels.csv', index=False)

In [8]:
df

,filename,maskedeye,age,pdays,label
0,al0320_2005_02_16_os.jpg,al0320l,48.416153,0,n
1,al0320_2006_01_13_os.jpg,al0320l,49.322382,331,n
2,al0320_2007_03_26_os.jpg,al0320l,50.518823,768,n
3,al0320_2011_04_26_os_02.jpg,al0320l,54.603696,2260,n
4,al0320_2012_07_25_os_02.jpg,al0320l,55.852156,2716,n
...,...,...,...,...,...
21997,boykins_leland_color fundus_4-2-2012_12-34-41 ...,vip123711l,69.097878,0,n
21998,boykins_leland_color fundus_4-2-2012_12-34-41 ...,vip123711r,69.097878,0,n
21999,1.2.826.0.1.3680043.2.139.2.1.206247280172.120...,vip123728r,80.312115,0,n
22000,rapley_matthew_color fundus_6-18-2014_1-27-02 ...,vip123795l,16.205339,0,b
